In [51]:
import io
import json
import pandas
from postgis import Point
from postgis.psycopg import register
import psycopg2
import requests

In [52]:
r = requests.get('http://web-services.unavco.org/internalWS/gps/metadata/stationcoordinate/sites/beta?project=PBO&refframe=igs08&verboseheader=true&stddev=false')

# set engine to python for regex separator
df = pandas.read_csv(io.StringIO(r.text), skiprows=6, header=None, names=['id', 'stationName', 'lat', 'lon',
       'elev', 'epoch', 'referenceFrame'], sep='\s*,\s*', engine='python')
#print(len(['data']))

In [59]:
r2 = requests.get('http://www.unavco.org/gsacws/gsacapi/site/search?output=site.csv&site.group=PBO+Analysis+Complete&site.sortorder=ascending&limit=99999')
df2 = pandas.read_csv(io.StringIO(r2.text), skiprows=1, header=None,
  names=['id', 'stationName', 'lat', 'lon', 'elev', 'startTime', 'stopTime'],sep='\s*,\s*', engine='python')
df2 = df2.drop(['lat', 'lon', 'elev'], axis=1)

In [60]:
merged_df = pandas.merge(df, df2, on=['id', 'stationName'])
print(merged_df.columns)

Index(['id', 'stationName', 'lat', 'lon', 'elev', 'epoch', 'referenceFrame',
       'startTime', 'stopTime'],
      dtype='object')


In [66]:
if merged_df.size > 0:
    connection = psycopg2.connect("dbname=geoserver user=etlfires")  
    register(connection)
    cursor = connection.cursor()
    cursor.execute("DELETE FROM {}".format('unavco_sites'))

    for idx, row in merged_df.iterrows():
        #print(site['siteLongitude'])
        cursor.execute("""
INSERT INTO unavco_sites (id, stationName, epoch, referenceFrame, startTime, stopTime, geom)
VALUES (%s, %s, %s, %s, %s, %s, %s)""",
            [row['id'], row['stationName'], row['epoch'] + 'Z', row['referenceFrame'],
            row['startTime'] + 'Z', row['stopTime'] + 'Z',
            Point(x=row['lon'], y=row['lat'], z=row['elev'], srid=4326)])
        
    connection.commit()
    connection.close()

In [ ]:
#list(d['data'][0].keys())

In [ ]:
#for i in range(0, len(d['data'])):
#    print('{} {} {}'.format(d['data'][i]['d['data'][i]['siteLatitude'], d['data'][i]['siteLongitude']))

In [ ]:
#d['data'][0]

In [ ]:
#type(d)

In [ ]:
#df.columns

In [ ]:
#for idx, row in df.iterrows():
#    print(row['ID'])

In [47]:
print(df.columns)
print(df2.columns)

Index(['id', 'stationName', 'lat', 'lon', 'elev', 'epoch', 'referenceFrame'], dtype='object')
Index(['id', 'stationName', 'start_time', 'stop_time'], dtype='object')


In [64]:
print(df[df['id'] == 'AB18'])
print(df2[df2['id'] == 'AB18'])

      id       stationName        lat         lon     elev  \
13  AB18  Kotzebue__AK2007  66.858361 -162.613508  37.1278   

                  epoch referenceFrame  
13  2017-10-25T00:03:27          IGS08  
      id       stationName            startTime             stopTime
17  AB18  Kotzebue__AK2007  2007-03-08T00:00:00  2017-11-20T23:59:45


In [48]:
merged_df = pandas.merge(df, df2, on=['id', 'stationName'])

In [49]:
merged_df.columns

Index(['id', 'stationName', 'lat', 'lon', 'elev', 'epoch', 'referenceFrame',
       'start_time', 'stop_time'],
      dtype='object')

In [50]:
print(merged_df)

        id       stationName        lat         lon       elev  \
0     AB01  AtkaIslandAK2007  52.209502 -174.204757    25.4878   
1     AB02  Nikolski__AK2007  52.970604 -168.854671   192.8047   
2     AB04  Savoonga__AK2007  63.656863 -170.567443   136.5619   
3     AB06  FalsePass_AK2005  54.885321 -163.423455   500.4354   
4     AB07  SandPoint_AK2004  55.349275 -160.476759    89.5786   
5     AB08  Mekoryuk__AK2008  60.384830 -166.200857    26.5176   
6     AB09  Razorback_AK2007  65.614979 -168.062125   162.0905   
7     AB11  Nome_AnvilAK2006  64.564494 -165.373459   349.4506   
8     AB12  Platinum__AK2007  58.950793 -161.746263   587.1021   
9     AB13  ChignikLgnAK2006  56.307325 -158.503796   487.8254   
10    AB14  DillinghamAK2007  59.108164 -159.091532   657.6529   
11    AB15  Nyac_Gold_AK2006  61.039751 -159.878362   560.3420   
12    AB17  UnalakleetAK2008  63.886358 -160.694732   117.4480   
13    AB18  Kotzebue__AK2007  66.858361 -162.613508    37.1278   
14    AB21